In [3]:
import numpy as np
import pandas as pd

data = pd.read_csv("./DATA/training.csv", encoding='ISO8859-2')
test = pd.read_csv("./DATA/test.csv", encoding="ISO8859-2")

In [4]:
from imblearn.under_sampling import RandomUnderSampler
data_some =data[['comment_text', 'identity_hate']]
rus = RandomUnderSampler()
X, y= rus.fit_resample(data[['comment_text']], data[['toxic']])
X

,comment_text
0,Hawaii five 0 \n\nI saw you deleted list of ha...
1,hey we are def hangin out noww!!! later!!
2,I found this article very easy to read and int...
3,Daily Closing Values \nDoes any one know: whe...
4,Well done. Forced mergers is how the banking ...
...,...
24359,Yeah i no it sucks.
24360,\nThank you. Given the misuse of tools here an...
24361,\n\n our previous conversation \n\nyou fucking...
24362,\n\nHey listen don't you ever!!!! Delete my ed...


,comment_text
0,\n\n Ethnic Taiwanese Imperial Japan Servicema...
1,|listas = Niels of Denmark
2,Why should anyone do such a thing?
3,:::::If you're offended by the Birther analogy...
4,"\n\nCooldown blocks are discouraged, aren't th..."
...,...
24359,Yeah i no it sucks.
24360,\nThank you. Given the misuse of tools here an...
24361,\n\n our previous conversation \n\nyou fucking...
24362,\n\nHey listen don't you ever!!!! Delete my ed...


In [5]:
data['comment_text_sw'] = data['comment_text'].apply(
    lambda text: " ".join([w for w in text.split()]))
test['comment_text_sw'] = test['comment_text'].apply(
    lambda text: " ".join([w for w in text.split()]))

In [6]:
result = test['id']
result

0        00054a5e18b50dd4
1        00078f8ce7eb276d
2        000897889268bc93
3        0009801bd85e5806
4        000c6a3f0cd3ba8e
               ...       
31909    ffba5332d6b8fd14
31910    ffbcd64a71775e04
31911    ffcdcb71854f6d8a
31912    ffe029a7c79dc7fe
31913    ffe987279560d7ff
Name: id, Length: 31914, dtype: object

In [7]:
# toxic 得分
from imblearn.under_sampling import RandomUnderSampler
rus = RandomUnderSampler()
X, y= rus.fit_resample(data[['comment_text_sw']], data[['toxic']])

from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(min_df=10)
X_tfidf = tfidf.fit_transform(X['comment_text_sw'])
from sklearn.decomposition import PCA
X_df = pd.DataFrame(X_tfidf.toarray(), columns=tfidf.get_feature_names_out())
pca = PCA(n_components=50)
x_pca = pca.fit_transform(X_df)
x_pca = pd.DataFrame(x_pca)
from sklearn.model_selection import train_test_split
X_train, X_valid, y_train, y_valid = train_test_split(x_pca, y, test_size=0.2)
from lightgbm.sklearn import LGBMClassifier
from sklearn.metrics import f1_score
lgb =LGBMClassifier(max_depth=10, num_leaves=100, boosting_type='dart')
lgb.fit(X_train, y_train)
y_valid_pred = lgb.predict(X_valid)
print("%s f1_score: %.2f" % ("toxic", f1_score(y_valid, y_valid_pred) * 100))

D:\anaconda\lib\site-packages\sklearn\preprocessing\_label.py:98: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
D:\anaconda\lib\site-packages\sklearn\preprocessing\_label.py:133: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


toxic f1_score: 82.90


In [8]:
test_tfidf = tfidf.transform(test['comment_text_sw'])
test_df = pd.DataFrame(test_tfidf.toarray(), columns=tfidf.get_feature_names_out())
test_pca = pca.transform(test_df)
test_pca = pd.DataFrame(test_pca)
toxic = lgb.predict(test_pca)
result = pd.concat([result, pd.DataFrame(toxic, columns=['toxic'], index=test.index)], axis=1)
result

,id,toxic
0,00054a5e18b50dd4,0
1,00078f8ce7eb276d,0
2,000897889268bc93,0
3,0009801bd85e5806,0
4,000c6a3f0cd3ba8e,0
...,...,...
31909,ffba5332d6b8fd14,0
31910,ffbcd64a71775e04,0
31911,ffcdcb71854f6d8a,0
31912,ffe029a7c79dc7fe,0


In [9]:
# severe_toxic 得分
from imblearn.under_sampling import RandomUnderSampler
rus = RandomUnderSampler()
X, y= rus.fit_resample(data[['comment_text_sw']], data[['severe_toxic']])

from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(min_df=10)
X_tfidf = tfidf.fit_transform(X['comment_text_sw'])
from sklearn.decomposition import PCA
X_df = pd.DataFrame(X_tfidf.toarray(), columns=tfidf.get_feature_names_out())
pca = PCA(n_components=50)
x_pca = pca.fit_transform(X_df)
x_pca = pd.DataFrame(x_pca)
from sklearn.model_selection import train_test_split
X_train, X_valid, y_train, y_valid = train_test_split(x_pca, y, test_size=0.2)
from lightgbm.sklearn import LGBMClassifier
from sklearn.metrics import f1_score
lgb =LGBMClassifier(max_depth=10, num_leaves=100, boosting_type='dart')
lgb.fit(X_train, y_train)
y_valid_pred = lgb.predict(X_valid)
print("%s f1_score: %.2f" % ("severe_toxic", f1_score(y_valid, y_valid_pred) * 100))

D:\anaconda\lib\site-packages\sklearn\preprocessing\_label.py:98: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
D:\anaconda\lib\site-packages\sklearn\preprocessing\_label.py:133: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


severe_toxic f1_score: 91.44


In [10]:
test_tfidf = tfidf.transform(test['comment_text_sw'])
test_df = pd.DataFrame(test_tfidf.toarray(), columns=tfidf.get_feature_names_out())
test_pca = pca.transform(test_df)
test_pca = pd.DataFrame(test_pca)
severe_toxic = lgb.predict(test_pca)
result = pd.concat([result, pd.DataFrame(severe_toxic, columns=['severe_toxic'], index=test.index)], axis=1)
result

,id,toxic,severe_toxic
0,00054a5e18b50dd4,0,0
1,00078f8ce7eb276d,0,0
2,000897889268bc93,0,0
3,0009801bd85e5806,0,0
4,000c6a3f0cd3ba8e,0,0
...,...,...,...
31909,ffba5332d6b8fd14,0,0
31910,ffbcd64a71775e04,0,0
31911,ffcdcb71854f6d8a,0,0
31912,ffe029a7c79dc7fe,0,0


In [11]:
# obscene 得分
from imblearn.under_sampling import RandomUnderSampler
rus = RandomUnderSampler()
X, y= rus.fit_resample(data[['comment_text_sw']], data[['obscene']])

from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(min_df=10)
X_tfidf = tfidf.fit_transform(X['comment_text_sw'])
from sklearn.decomposition import PCA
X_df = pd.DataFrame(X_tfidf.toarray(), columns=tfidf.get_feature_names_out())
pca = PCA(n_components=50)
x_pca = pca.fit_transform(X_df)
x_pca = pd.DataFrame(x_pca)
from sklearn.model_selection import train_test_split
X_train, X_valid, y_train, y_valid = train_test_split(x_pca, y, test_size=0.2)
from lightgbm.sklearn import LGBMClassifier
from sklearn.metrics import f1_score
lgb =LGBMClassifier(max_depth=10, num_leaves=100, boosting_type='dart')
lgb.fit(X_train, y_train)
y_valid_pred = lgb.predict(X_valid)
print("%s f1_score: %.2f" % ("obscene", f1_score(y_valid, y_valid_pred) * 100))

D:\anaconda\lib\site-packages\sklearn\preprocessing\_label.py:98: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
D:\anaconda\lib\site-packages\sklearn\preprocessing\_label.py:133: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


obscene f1_score: 86.99


In [12]:
test_tfidf = tfidf.transform(test['comment_text_sw'])
test_df = pd.DataFrame(test_tfidf.toarray(), columns=tfidf.get_feature_names_out())
test_pca = pca.transform(test_df)
test_pca = pd.DataFrame(test_pca)
obscene = lgb.predict(test_pca)
result = pd.concat([result, pd.DataFrame(obscene, columns=['obscene'], index=test.index)], axis=1)
result

,id,toxic,severe_toxic,obscene
0,00054a5e18b50dd4,0,0,0
1,00078f8ce7eb276d,0,0,0
2,000897889268bc93,0,0,0
3,0009801bd85e5806,0,0,0
4,000c6a3f0cd3ba8e,0,0,0
...,...,...,...,...
31909,ffba5332d6b8fd14,0,0,0
31910,ffbcd64a71775e04,0,0,0
31911,ffcdcb71854f6d8a,0,0,0
31912,ffe029a7c79dc7fe,0,0,0


In [13]:
# threat 得分
from imblearn.under_sampling import RandomUnderSampler
rus = RandomUnderSampler()
X, y= rus.fit_resample(data[['comment_text_sw']], data[['threat']])

from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(min_df=10)
X_tfidf = tfidf.fit_transform(X['comment_text_sw'])
from sklearn.decomposition import PCA
X_df = pd.DataFrame(X_tfidf.toarray(), columns=tfidf.get_feature_names_out())
pca = PCA(n_components=50)
x_pca = pca.fit_transform(X_df)
x_pca = pd.DataFrame(x_pca)
from sklearn.model_selection import train_test_split
X_train, X_valid, y_train, y_valid = train_test_split(x_pca, y, test_size=0.2)
from lightgbm.sklearn import LGBMClassifier
from sklearn.metrics import f1_score
lgb =LGBMClassifier(max_depth=10, num_leaves=100, boosting_type='dart')
lgb.fit(X_train, y_train)
y_valid_pred = lgb.predict(X_valid)
print("%s f1_score: %.2f" % ("threat", f1_score(y_valid, y_valid_pred) * 100))

D:\anaconda\lib\site-packages\sklearn\preprocessing\_label.py:98: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
D:\anaconda\lib\site-packages\sklearn\preprocessing\_label.py:133: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


threat f1_score: 85.37


In [14]:
test_tfidf = tfidf.transform(test['comment_text_sw'])
test_df = pd.DataFrame(test_tfidf.toarray(), columns=tfidf.get_feature_names_out())
test_pca = pca.transform(test_df)
test_pca = pd.DataFrame(test_pca)
threat = lgb.predict(test_pca)
result = pd.concat([result, pd.DataFrame(threat, columns=['threat'], index=test.index)], axis=1)
result

,id,toxic,severe_toxic,obscene,threat
0,00054a5e18b50dd4,0,0,0,0
1,00078f8ce7eb276d,0,0,0,0
2,000897889268bc93,0,0,0,0
3,0009801bd85e5806,0,0,0,0
4,000c6a3f0cd3ba8e,0,0,0,0
...,...,...,...,...,...
31909,ffba5332d6b8fd14,0,0,0,0
31910,ffbcd64a71775e04,0,0,0,0
31911,ffcdcb71854f6d8a,0,0,0,0
31912,ffe029a7c79dc7fe,0,0,0,0


In [15]:
# insult 得分
from imblearn.under_sampling import RandomUnderSampler
rus = RandomUnderSampler()
X, y= rus.fit_resample(data[['comment_text_sw']], data[['insult']])

from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(min_df=10)
X_tfidf = tfidf.fit_transform(X['comment_text_sw'])
from sklearn.decomposition import PCA
X_df = pd.DataFrame(X_tfidf.toarray(), columns=tfidf.get_feature_names_out())
pca = PCA(n_components=50)
x_pca = pca.fit_transform(X_df)
x_pca = pd.DataFrame(x_pca)
from sklearn.model_selection import train_test_split
X_train, X_valid, y_train, y_valid = train_test_split(x_pca, y, test_size=0.2)
from lightgbm.sklearn import LGBMClassifier
from sklearn.metrics import f1_score
lgb =LGBMClassifier(max_depth=10, num_leaves=100, boosting_type='dart')
lgb.fit(X_train, y_train)
y_valid_pred = lgb.predict(X_valid)
print("%s f1_score: %.2f" % ("insult", f1_score(y_valid, y_valid_pred) * 100))

D:\anaconda\lib\site-packages\sklearn\preprocessing\_label.py:98: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
D:\anaconda\lib\site-packages\sklearn\preprocessing\_label.py:133: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


insult f1_score: 86.90


In [16]:
test_tfidf = tfidf.transform(test['comment_text_sw'])
test_df = pd.DataFrame(test_tfidf.toarray(), columns=tfidf.get_feature_names_out())
test_pca = pca.transform(test_df)
test_pca = pd.DataFrame(test_pca)
insult = lgb.predict(test_pca)
result = pd.concat([result, pd.DataFrame(insult, columns=['insult'], index=test.index)], axis=1)
result

,id,toxic,severe_toxic,obscene,threat,insult
0,00054a5e18b50dd4,0,0,0,0,0
1,00078f8ce7eb276d,0,0,0,0,0
2,000897889268bc93,0,0,0,0,0
3,0009801bd85e5806,0,0,0,0,0
4,000c6a3f0cd3ba8e,0,0,0,0,0
...,...,...,...,...,...,...
31909,ffba5332d6b8fd14,0,0,0,0,0
31910,ffbcd64a71775e04,0,0,0,0,0
31911,ffcdcb71854f6d8a,0,0,0,0,0
31912,ffe029a7c79dc7fe,0,0,0,0,0


In [17]:
# identity_hate 得分
from imblearn.under_sampling import RandomUnderSampler
rus = RandomUnderSampler()
X, y= rus.fit_resample(data[['comment_text_sw']], data[['identity_hate']])

from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(min_df=10)
X_tfidf = tfidf.fit_transform(X['comment_text_sw'])
from sklearn.decomposition import PCA
X_df = pd.DataFrame(X_tfidf.toarray(), columns=tfidf.get_feature_names_out())
pca = PCA(n_components=50)
x_pca = pca.fit_transform(X_df)
x_pca = pd.DataFrame(x_pca)
from sklearn.model_selection import train_test_split
X_train, X_valid, y_train, y_valid = train_test_split(x_pca, y, test_size=0.2)
from lightgbm.sklearn import LGBMClassifier
from sklearn.metrics import f1_score
lgb =LGBMClassifier(max_depth=10, num_leaves=100, boosting_type='dart')
lgb.fit(X_train, y_train)
y_valid_pred = lgb.predict(X_valid)
print("%s f1_score: %.2f" % ("identity_hate", f1_score(y_valid, y_valid_pred) * 100))

D:\anaconda\lib\site-packages\sklearn\preprocessing\_label.py:98: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
D:\anaconda\lib\site-packages\sklearn\preprocessing\_label.py:133: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


identity_hate f1_score: 83.18


In [18]:
test_tfidf = tfidf.transform(test['comment_text_sw'])
test_df = pd.DataFrame(test_tfidf.toarray(), columns=tfidf.get_feature_names_out())
test_pca = pca.transform(test_df)
test_pca = pd.DataFrame(test_pca)
identity_hate = lgb.predict(test_pca)
result = pd.concat([result, pd.DataFrame(identity_hate, columns=['identity_hate'], index=test.index)], axis=1)
result

,id,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,00054a5e18b50dd4,0,0,0,0,0,0
1,00078f8ce7eb276d,0,0,0,0,0,0
2,000897889268bc93,0,0,0,0,0,0
3,0009801bd85e5806,0,0,0,0,0,0
4,000c6a3f0cd3ba8e,0,0,0,0,0,0
...,...,...,...,...,...,...,...
31909,ffba5332d6b8fd14,0,0,0,0,0,0
31910,ffbcd64a71775e04,0,0,0,0,0,0
31911,ffcdcb71854f6d8a,0,0,0,0,0,0
31912,ffe029a7c79dc7fe,0,0,0,0,0,0


In [19]:
result.to_csv("_results.csv", index=None)